In [1]:
import os
import pandas as pd
from datetime import date
from datetime import datetime
import numpy as np
import seaborn as sns
from scipy import stats
from glob import glob
import re
%config Completer.use_jedi = False
from functools import reduce

## Load Data from REDCap Table

In [2]:
csv_dir = '/Volumes/NBL_Projects/NSF_SNP/matfiles/' +\
          'preprocessing_pipeline_surface_analyses/Analysis_in_Python/spreadsheets/'
csv = 'DevelopmentOfSymboli_DATA_2021-11-08_1321.csv'
df = pd.read_csv(csv_dir + csv)
df = df.set_index(['record_id'])

## Transform the REDCap table to combine across event types and merge across visits

In [3]:
# Create dictionary of dataframes broken down by year (i.e., "1" appears in event names for Y1)
dfs = {0:df.query("redcap_event_name.str.contains('1')",engine='python'),
       1:df.query("redcap_event_name.str.contains('2')",engine='python'),
       2:df.query("redcap_event_name.str.contains('3')",engine='python')}

visit_code = ('_Y1','_Y2','_Y3')

# Combine data across event types, within each year (i.e., visit)
# For dataframes (dfs) separated by each year 
for i in dfs:
    # Separate by event type
    # Dictionary with separate dataframe for each event type
    d = dict(tuple(dfs[i].groupby('redcap_event_name')))
    # List of event names (dict keys)
    ls = list(d)
    # Create combined dataset, filling in missing values from each event
    # Start with first event type as the base 
    dout = d[ls[0]]
    # Loop through remaining events
    for l in ls[1:]:
        # Create combined dataset, filling in missing values from each event
        dout = dout.combine_first(d[l])
    # Add year suffix, and replace df in original dictionary
    dfs[i] = dout.add_suffix(visit_code[i])
    
# Merge the dataframes from each year     
# Function to merge multiple input dataframes (as a single dictionary)
def merge_dfs(dfList,index_col):
    temp = reduce(lambda left, right: pd.merge(left, right, how='outer', on=index_col), dfList)
    return temp
# Run merge function
df = merge_dfs(dfs.values(),'record_id')


/usr/local/lib/python3.9/site-packages/pandas/core/reshape/merge.py:120: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  return op.get_result()


## Load scan session trial information

In [4]:
# Get information about task trials and accuracy from AFNI-created event tables
# Though REDCap has (EPrime-generated) accuracy for task and runs, the AFNI 
# event tables reflect the actual data that went into fMRI processing.
# The event tables also allow us to easily get the number of trials for binomial testing.

# Find all the event tables within the Processed_Data directory
event_tbls = glob('/Volumes/NBL_Projects/NSF_SNP/Processed_Data/SNP*/' 
                  '/Y*/SYM/proc/onsets/SNP*SYM*event_table.csv')
for e in event_tbls:
    tmp = pd.read_csv(e)
    # Hardcode event list (for SYM task, three conditions of interest = Dig, Let, Scr)
    events = [1,2,3]
    # Get total number of events from specified event list
    # *Note that this = number correct, since errors and omissions 
    # are counted as different events (i.e., class 4 and 5)
    ncorrect = 0
    for c in events: 
        ncorrect += (tmp.EventClass == c).sum()
    ntrials = len(tmp)
    # Compute accuracy
    acc = ncorrect/ntrials
    # Check if this is significantly above chance accuracy (alpha = 0.05) using binomial test
    abv_chance = stats.binom_test(ncorrect,ntrials,p=0.5,alternative='greater') < 0.05
    
    # Get subject and visit code from the file name
    subj = re.search('SNP\d+',e)[0]
    year = re.search('Y\d',e)[0]

    # Add to main dataframe
    df.loc[subj,'afni_acc_SYM_' + year] = acc
    df.loc[subj,'afni_ntrials_SYM_' + year] = ntrials
    df.loc[subj,'afni_ncorrect_SYM_' + year] = ncorrect
    df.loc[subj,'afni_abv_chance_SYM_' + year] = abv_chance


/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


## Define function to compute days in school based on visit date and grade

In [5]:
# Compute days in school based on input date (e.g. scan date) and grade
def calculate_days_in_school(date_in,grade):
    if pd.isna(date_in) | pd.isna(grade):
        return np.nan
    else:
        # Convert input data to proper format
        grade = int(grade)
        date_in = date_in.to_pydatetime().date()

        # Set default begining and end date of school year
        beg = [8,10]
        end = [5,20]
        
        # Get year of input date
        date_year = date_in.year
        
        # Define school beginning and end dates relative to input date
        school_beg = date(date_year, beg[0], beg[1])
        school_end = date(date_year, end[0], end[1])
        school_beg_prev = date(date_year-1, beg[0], beg[1])

        # Define total number of days in academic year (note could change if leap year)
        school_days_year = school_end - school_beg_prev 

        # Logic to calculate total number of days in school, check in Fall, Summer, or Spring
        if date_in >= school_beg:  # Date during Fall
            total = (grade*school_days_year) + (date_in-school_beg)
        elif date_in > school_end: # Date during Summer
            total = (grade+1)*school_days_year
        else:                       # Date during Spring
            total = ((grade)*school_days_year) + (date_in-school_beg_prev)

        return total

## Create multiple derived variables

In [6]:
# Set date columns to datetime format
for col in df.filter(regex='date|dob').columns:
    df[col] = pd.to_datetime(df[col])

# Calculate age of child at scan and behavioral testing
for y in visit_code:
    df['age_scan' + y] = df['scandate' + y] - df['child_dob' + y]
    df['age_beh' + y] =  df['date_of_testing' + y] - df['child_dob' + y]

# Calculate total days in school (for each visit)
for y in visit_code:
    df['tot_school_days' + y] = df.apply(lambda row: calculate_days_in_school(
                                    row['scandate' + y], row['grade' + y]), axis = 1)

# Calculate mean accuracy from REDCap entries per run (just for verification vs AFNI event tables)
for y in visit_code:
    df['sym_acc_RdCp' + y] = df.filter(regex='sym_r.*_acc_all' + y).mean(axis=1,skipna=True)

# Compute usability variable based on above chance + censor fraction at most 25%
def is_usable(abv_chance, censorfrac):
    if abv_chance and censorfrac <= 0.25:
        usable = True
    else:
        usable = False
    return usable
# Apply to each visit timepoint
for y in visit_code:
    df['sym_usable' + y] = df.apply(lambda x: is_usable(x['afni_abv_chance_SYM' + y],
                                                        x['sym_censorfrac' + y]), axis=1)

# Add visit code as an explicit variable
# Helps keep track of these rows in stacked table and 
# is useful for associating imaging data later on
for y in visit_code:
    df['visit_code' + y] = y.replace('_','')

## Load the imaging data
#### Build a multindex dataframe, with subject as first level, then the contrast_hemi_visit at second level. The columns will refer to each of the nodes on the surface (e.g. 36002 columns for AFNI's ld60 mesh)


In [7]:
# Set hemispheres and contrasts of interest
hemi = ['lh','rh']
briks = ['Dig#0_Coef',   'Dig#0_Tstat',
         'Let#0_Coef',   'Let#0_Tstat',
         'Scr#0_Coef',   'Scr#0_Tstat',
         'error#0_Coef', 'error#0_Tstat',
         'omiss#0_Coef', 'omiss#0_Tstat',
         'D-L#0_Coef',   'D-L#0_Tstat',
         'D-S#0_Coef',   'D-S#0_Tstat',
         'D-L+S#0_Coef', 'D-L+S#0_Tstat',
         'L-S#0_Coef',   'L-S#0_Tstat',
         'L-D+S#0_Coef', 'L-D+S#0_Tstat',
         'Fml#0_Coef',   'Fml#0_Tstat',
         'Fml-Scr#0_Coef','Fml-Scr#0_Tstat',
         'Correct#0_Coef','Correct#0_Tstat',
         'Incorrect#0_Coef','Incorrect#0_Tstat',
        ]


# Function to extract specific "briks" (i.e. statistical contrasts) from surface niml.dset files
def load_surface_data(f,briks):
    f_pre = f.replace('.niml.dset','')
    f_1D = f.replace('.niml.dset','.1D.dset')
    
    # Convert the niml.dset file if needed
    if not os.path.exists(f_1D):
        !ConvertDset -overwrite -o_1Dp -input {f} -prefix {f_pre}
    
    # Get the brik indices of stats of interest
    tmp_file = 'tmp_brik.txt'
    !rm -f {tmp_file}
    for b in briks:
        # Get the brik index from the label (note AFNI index starts at 0, consistent with Python)
        !3dinfo -label2index {b} {f} >> {tmp_file}
    brik_inds = np.loadtxt(tmp_file)
    !rm -f {tmp_file}
    
    # Load only the columns (brik) of interest from the dataset
    try: 
        out = np.loadtxt(f_1D,usecols=brik_inds.astype(int))
    except: # If file won't load, attempt to recreate it first
        !ConvertDset -overwrite -o_1Dp -input {f} -prefix {f_pre}
        out = np.loadtxt(f_1D,usecols=brik_inds.astype(int))
        
    return out


# Loop through hemispheres, building the full dataframe
df_surf = pd.DataFrame()
for h in hemi:
    # Get filenames for this hemishphere
    fnames = glob('/Volumes/NBL_Projects/NSF_SNP/Processed_Data/SNP*/'
                    'Y*/SYM/proc/SNP*_SYM_Y*.surface.univariate/'
                    f'stats.SNP*_SYM_Y*.{h}_REML.niml.dset')
    # Loop through files
    for f in fnames:
        # Get the subject name and visit code from the filename
        subj = re.search('SNP\d+',f)[0]
        year = re.search('Y\d',f)[0]
        print(f'{subj} {year} {h}')
        # Load the niml data into numpy array
        out = load_surface_data(f,briks)
        # Set the labels for these data
        # For readibility, remove the '#0' that AFNI inserts
        briks_clean = [b.replace('#0', '') for b in briks]
        labels = [f'{b}_{h}_{year}' for b in briks_clean]
        # Specify multindex for temporary dataframe 
        # 1st level = subject
        # 2nd level = contrast/hemi/visit
        idx = pd.MultiIndex.from_product([[subj],labels],names=['subject', 'contrast'])
        # Append temporary dataframe to full dataframe
        df_surf = df_surf.append(pd.DataFrame(data=out.transpose(),index=idx))
df_surf

SNP001 Y2 lh
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFN

++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
SNP005 Y2 lh
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFN

++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
SNP007 Y2 lh
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFN

++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
SNP010 Y3 lh
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFN

++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
SNP013 Y3 lh
++ 3dinfo: AFNI version=AFN

...

++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Ju

SNP103 Y2 rh
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFN

++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
SNP108 Y2 rh
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFN

++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]
++ 3dinfo: AFNI version=AFNI_20.1.12 (Jun  1 2020) [64-bit]


0         1         2         3      \
subject contrast                                                        
SNP001  Dig_Coef_lh_Y2        -0.206946 -0.287306  0.247002 -0.301592   
        Dig_Tstat_lh_Y2       -0.876085 -1.855345  1.334751 -1.736867   
        Let_Coef_lh_Y2         0.024009 -0.097165  0.256246 -0.238690   
        Let_Tstat_lh_Y2        0.103413 -0.634327  1.403473 -1.394349   
        Scr_Coef_lh_Y2        -0.124149 -0.179017 -0.042711  0.095947   
...                                 ...       ...       ...       ...   
SNP110  Fml-Scr_Tstat_rh_Y2    0.174324  0.159686  0.724982 -4.551522   
        Correct_Coef_rh_Y2    -1.932173 -0.210274 -0.466133  0.659864   
        Correct_Tstat_rh_Y2   -1.039105 -0.613059 -1.166427  1.878138   
        Incorrect_Coef_rh_Y2   0.742508  1.239213 -0.979694  0.000057   
        Incorrect_Tstat_rh_Y2  0.277154  1.891268 -1.719418  0.000092   

                                  4         5         6         7      \
subject contrast                                                        
SNP001  Dig_Coef_lh_Y2        -0.032772 -0.092335  0.237935  0.186269   
        Dig_Tstat_lh_Y2       -0.156604 -0.501606  1.132674  0.977644   
        Let_Coef_lh_Y2         0.095877 -0.168233 -0.022821 -0.054253   
        Let_Tstat_lh_Y2        0.462395 -0.921226 -0.109254 -0.286666   
        Scr_Coef_lh_Y2        -0.076491 -0.073112  0.348112  0.027883   
...                                 ...       ...       ...       ...   
SNP110  Fml-Scr_Tstat_rh_Y2   -0.586795 -2.304510  0.554599 -0.490811   
        Correct_Coef_rh_Y2    -0.015339 -0.350016 -0.268163 -0.093773   
        Correct_Tstat_rh_Y2   -0.028847 -1.061958 -0.693645 -0.357712   
        Incorrect_Coef_rh_Y2  -0.165629 -0.368867 -0.793819 -0.073120   
        Incorrect_Tstat_rh_Y2 -0.168777 -0.682773 -1.216763 -0.157332   

                                  8         9      ...     35992     35993  \
subject contrast                                   ...                       
SNP001  Dig_Coef_lh_Y2        -0.017437  0.376099  ...  0.225704  0.190759   
        Dig_Tstat_lh_Y2       -0.113274  1.819653  ...  0.877271  0.838654   
        Let_Coef_lh_Y2        -0.183114  0.370434  ...  0.515701  0.449724   
        Let_Tstat_lh_Y2       -1.206635  1.815037  ...  2.034605  2.008812   
        Scr_Coef_lh_Y2         0.220899  0.538034  ...  0.545266  0.424869   
...                                 ...       ...  ...       ...       ...   
SNP110  Fml-Scr_Tstat_rh_Y2    0.382538 -1.248977  ... -1.029595 -0.932688   
        Correct_Coef_rh_Y2    -0.305445 -1.370130  ...  0.240700  0.047466   
        Correct_Tstat_rh_Y2   -0.980981 -2.670064  ...  0.843088  0.151976   
        Incorrect_Coef_rh_Y2   0.398764 -0.080249  ...  0.136198 -0.128090   
        Incorrect_Tstat_rh_Y2  0.870457 -0.103382  ...  0.303069 -0.259277   

                                  35994     35995     35996     35997  \
subject contrast                                                        
SNP001  Dig_Coef_lh_Y2        -0.025920 -0.219015  0.346205  0.179291   
        Dig_Tstat_lh_Y2       -0.120401 -0.760779  1.509291  0.788831   
        Let_Coef_lh_Y2         0.159362 -0.107933  0.500361  0.323919   
        Let_Tstat_lh_Y2        0.751408 -0.380563  2.214170  1.447963   
        Scr_Coef_lh_Y2         0.019211 -0.344289  0.562233  0.299002   
...                                 ...       ...       ...       ...   
SNP110  Fml-Scr_Tstat_rh_Y2   -0.234492  0.287217 -1.122492 -0.445140   
        Correct_Coef_rh_Y2     0.106084  0.485461  0.169729  0.029249   
        Correct_Tstat_rh_Y2    0.304708  1.152916  0.627768  0.095394   
        Incorrect_Coef_rh_Y2  -0.185161 -0.226847  0.085941 -0.104366   
        Incorrect_Tstat_rh_Y2 -0.320600 -0.319243  0.201938 -0.209977   

                                  35998     35999     36000     36001  
subject contrast                                                       
SNP001  Dig_Coef_lh_Y2   

## Write tables to csv files

In [8]:
# Write to CSV
df.to_csv('SNP_REDCap_Export_and_Derived_Vars.csv',na_rep='NaN')
df_surf.to_csv('SNP_Activation_Stats_on_Surface_Nodes.csv',na_rep='NaN')


## Stack table in preparation for modeling

In [9]:
# Set the variables to keep in the final stacked table
# Note, you could add any (e.g., behavioral) variable(s) from REDCap 
vars_keep = ['visit_code',
             'tot_school_days',
             'sym_usable',
             'sym_censorfrac',
             'age_scan',
             'child_gender',
             'child_handedness']

df_stack = pd.DataFrame()
# Loop through visit codes
for y in visit_code:
    # Loop through each variable name and grad column with appended visit code
    df_tmp = df[[vk + y for vk in vars_keep]]
    df_tmp.columns = vars_keep
    df_stack = df_stack.append(df_tmp)

df_stack.to_csv('Test_StackTable.csv',na_rep='NaN')

In [10]:
# Dataframe multi-index slicing examples
# https://stackoverflow.com/questions/53927460/select-rows-in-pandas-multiindex-dataframe
# df.loc[['SNP001']]
# df.query("redcap_event_name == 'mri_y1_arm_1'")

# items = ['SNP001', 'SNP002']
# df.query("record_id in @items")

# items = ['mri_y1_arm_1','mri_y2_arm_2','mri_y3_arm_3']
# df_test = df.query("redcap_event_name.str.contains('1')",engine='python')
# df_test.head(-1)
# df.xs(('SNP001', 'mri_y1_arm_1')